In [1]:
import numpy as np
from shapely.geometry import Point, LineString


class TheRightRoute():
    def __init__(self, waypoints: LineString, acceptable_distance):
        self.route = waypoints
        self.acceptable_distance = acceptable_distance
        
        self.start_pt = Point(self.route.coords[0])
        self.finish_line = self.calc_finish_line()
    
    def calc_finish_line(self):
        if self.route.coords[-1][0]-self.route.coords[-2][0] == 0:
            RouteEndAngle = np.pi /2
        else:
            RouteEndAngle = np.arctan((self.route.coords[-1][1]-self.route.coords[-2][1])/ \
                                      (self.route.coords[-1][0]-self.route.coords[-2][0]))
        finish_line = LineString([
            Point(
                self.route.coords[-1][0] - acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] + acceptable_distance * np.cos(RouteEndAngle)),
            Point(
                self.route.coords[-1][0] + acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] - acceptable_distance * np.cos(RouteEndAngle))
        ])
        return finish_line
    
    

In [2]:
import numpy as np
%matplotlib inline

n_cars = 20
n_loops = 10
step_length = 1
max_steering = 45 * np.pi / 180 # 45deg in radians
acceptable_distance = 10


# RouteToFollow = LineString([(0,0), (60,100)]) # Straight line
RouteToFollow = LineString([(0,0), (60, 0), (60, 60)]) # L-shape
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0)]) # Bumpy road
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (75, 30), (90, 0), (120,0)]) # Bumpy road with shortcut
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0), (130,90), (10, 85), (10, 30)]) # Snail
# RouteToFollow = LineString([(0,0), (20, 0), (20, 30), (50, 30), (50, -30), (20, -30), (20,-60), (80,-60), (80, 60), \
#                             (-30, 60), (-30, 0), (0, -90), (110, -90), (110, 0), (130, 0)]) # Labyrinth

the_right_route = TheRightRoute(RouteToFollow, acceptable_distance)

n_steps = int(the_right_route.route.length*1.5)

In [20]:
import numpy as np
import pandas as pd

class FinderStrategy_1():
    
    def __init__(self, n_cars, n_steps, max_steering):
        self.n_cars = n_cars
        self.n_steps = n_steps
        self.max_steering = max_steering
    
    
    def generate_random_steering_angles(self, n_trajectories: int, length: int = None):
        if length is None:
            length = self.n_steps
        return [np.array([0] + list(self.max_steering*np.random.normal(0, 0.5, size=length))) \
                for i in range(n_trajectories)]
    
    
    def generate_twins(self, df_vehicles):
        v_best_score = df_vehicles.nlargest(n=1, columns='Score').iloc[0]
        base = v_best_score['Steering angles']
        n_adnl_combinations = self.n_cars - 1

        max_upto = int(v_best_score['TrajectoryLength'])
        shorten_halflength = int(0.05*v_best_score['TrajectoryLength'])
        list_keep_upto = max_upto - np.clip(np.abs(np.random.normal(0, max_upto/3.3, (n_adnl_combinations))), 0, max_upto).astype(int)

        output = [self.make_new_branch(base, keep_upto) if keep_upto > int(0.80*max_upto) \
                  else self.shorten_branch(base, v_best_score['Trajectory'], int(keep_upto-shorten_halflength), int(keep_upto+shorten_halflength)) \
                  for keep_upto in list_keep_upto]
        return [base] + output
    
    
    def make_new_branch(self, base, keep_upto):
        return np.concatenate((base[:keep_upto], self.generate_random_steering_angles(1, length=len(base)-keep_upto)[0]))
    
    
    def shorten_branch(self, base_angles, trajectory, rework_from, rework_to):
        deltaX = trajectory.coords[rework_to][0] - trajectory.coords[rework_from][0]
        deltaY = trajectory.coords[rework_to][1] - trajectory.coords[rework_from][1]
        n_steps_shortcut = int(np.round(np.sqrt(deltaX**2 + deltaY**2)))

        sum_prev_angles = np.sum(base_angles[:rework_from])
        ini_angle = np.sign(deltaY)*np.pi if deltaX == 0 else (np.arctan(deltaY/deltaX) - sum_prev_angles)
        finish_angle = np.sum(base_angles[:rework_to]) - sum_prev_angles - ini_angle

        new_length = rework_from + n_steps_shortcut + len(base_angles) - rework_to
        add_n_elements = rework_to - rework_from - n_steps_shortcut -1
        add_elements = [] if add_n_elements <= 0 else self.generate_random_steering_angles(1, 1+add_n_elements)[0][1:]

        new_angles = np.concatenate((base_angles[:rework_from], [ini_angle], np.zeros(n_steps_shortcut-2), \
                                     [finish_angle], base_angles[rework_to:], add_elements))[:len(base_angles)]
        return new_angles
    
    
    def calc_score(self, vehicle: pd.Series):
        return vehicle['DistanceFromStart'] - sum([abs(val) for val in vehicle['Steering angles']])
    

finder_strategy = FinderStrategy_1(n_cars, n_steps, max_steering)

In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import takewhile
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import ipywidgets as widgets
import geopandas as gpd

class PathFinder():
    def __init__(self, the_right_route, finder_strategy, acceptable_distance, n_cars, n_steps, n_loops, step_length):
        self.the_right_route = the_right_route
        self.finder_strategy = finder_strategy # Why not generate instance of strategy WITHIN here? (more safe re variables)
        
        self.acceptable_distance = acceptable_distance
        self.n_cars = n_cars
        self.n_steps = n_steps
        self.n_loops = n_loops
        self.step_length = step_length
        
        self.vehicles_history = None
    
    
    def run_simulation(self):
        steering_angles = self.finder_strategy.generate_random_steering_angles(self.n_cars)
        self.vehicles_history = pd.DataFrame()

        for loop in tqdm(range(self.n_loops+1)):
            vehicles = self.calc_vehicles_df(loop, steering_angles)
            self.vehicles_history = pd.concat([self.vehicles_history, vehicles])
            steering_angles = self.finder_strategy.generate_twins(vehicles) # Prep next iteration. MOST IMPORTANT STEP FOR LEARNING!
        return None
    
    
    def calc_vehicles_df(self, loop, steering_angles):
        vehicles = pd.DataFrame({'Loop': loop, 'Steering angles': steering_angles})
        vehicles['Trajectory'] = vehicles['Steering angles'].apply(self.calc_trajectory)
        vehicles['DistanceFromStart'] = vehicles['Trajectory'].apply(self.calc_distance_from_start)
        vehicles['TrajectoryLength'] = vehicles['Trajectory'].apply(lambda traj: traj.length)
        vehicles['Score'] = vehicles.apply(self.finder_strategy.calc_score, axis=1)
        return vehicles
    
    
    def calc_trajectory(self, steering_angles):
        steering_angles_cumsum = np.cumsum(steering_angles)
        Xpts = (self.step_length*np.cos(steering_angles_cumsum)).cumsum()
        Ypts = (self.step_length*np.sin(steering_angles_cumsum)).cumsum()
        
        trajectory = [self.the_right_route.start_pt] + list(takewhile(lambda pt: pt.distance(self.the_right_route.route) <= self.acceptable_distance, self.gen_points(Xpts, Ypts)))
        return LineString(trajectory)
    
    
    def gen_points(self, Xpts, Ypts):
        for pt in list(zip(Xpts, Ypts)):
            yield Point(pt[0], pt[1])
    
    
    def calc_distance_from_start(self, trajectory):
        return self.the_right_route.route.project(Point(list(trajectory.coords)[-1]))
    
    
    def display_interactive_plot(self):
        widgets.interact(self.plot_path, loop = widgets.IntSlider(
            value=0,
            min=0,
            max=self.n_loops,
            step=1,
            description='Loop:',
            disabled=False,
            orientation='horizontal',
            continuous_update=True,
            readout=True,
            readout_format='d',
            layout=widgets.Layout(width='95%')
        ));
    
    
    def plot_path(self, loop):
        vehicles = self.vehicles_history.loc[self.vehicles_history['Loop'] == loop]
        shapes_on_map = [{'geometry': self.the_right_route.start_pt, 'name': 'Start'},
                         {'geometry': self.the_right_route.finish_line, 'name': 'Finish'},
                         {'geometry': self.the_right_route.route.buffer(self.acceptable_distance, cap_style=2), 'name': 'Route to follow'}]
        vehicles.apply(lambda row: shapes_on_map.append({
            'geometry': row['Trajectory'],
            'name': f'Car{row.name}'
        }), axis=1)

        gdf = gpd.GeoDataFrame(shapes_on_map)
        ax = gdf.plot(column='name', figsize=(12,6), cmap='RdYlBu')#, legend=True)
        ax.set_title(f"Loop {loop} || Max score {vehicles['Score'].max():.2f} || Max dist from start {vehicles['DistanceFromStart'].max():.2f}", y=1.0, pad=-14, fontsize=8)

        dims = np.round(gdf.total_bounds*1.10)
        plt.xlim(dims[0], dims[2])
        plt.ylim(dims[1], dims[3])
        plt.grid()
        plt.show()

    
pf = PathFinder(the_right_route, finder_strategy, acceptable_distance, n_cars, n_steps, n_loops, step_length)

In [24]:
pf.run_simulation()

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:01<00:00,  9.86it/s]


In [25]:
pf.display_interactive_plot()

interactive(children=(IntSlider(value=0, description='Loop:', layout=Layout(width='95%'), max=10), Output()), …